<a href="https://colab.research.google.com/github/danangwijaya/JLH_Air/blob/main/jlh_air_colabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#INISIASI LIBRARY YANG DIGUNAKAN
from google.colab import drive
import glob
import os
# import leafmap
import geopandas as gp
import pandas as pds
import numpy as np
from datetime import datetime
import fiona
fiona.supported_drivers



{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [2]:
#INISIASI DATA YANG DIGUNAKAN
table_pl = pds.read_csv('/content/drive/MyDrive/Colab Notebooks/preprocessing/pl.csv')
ekoregion = gp.read_file('/content/drive/MyDrive/Colab Notebooks/preprocessing/Ekoregion/Ekoregion.shp')
ekoregion_proj = ekoregion.to_crs(3395)
PL_1996 = gp.read_file('/content/drive/MyDrive/Colab Notebooks/preprocessing/PL/1996/PL_1996.shp')
PL_2021 = gp.read_file('/content/drive/MyDrive/Colab Notebooks/preprocessing/PL/2021/PL_2021.shp')
# poly_prepros = gp.read_file(data_input)

#PROSES OVERLAY DATA
overlay_old = ekoregion_proj.overlay(PL_1996, how='intersection')
overlay_new = overlay_old.overlay(PL_2021, how='intersection')
jlh = gp.GeoDataFrame(overlay_new)

#PROSES JOIN TABLE SKOR PL
# jlh['pl'] = jlh['pl'].fillna(jlh.pop('PL20_ID'))
jlh_pl_old = gp.merge(jlh, table_pl, left_on='PL96_ID', right_on='pl',suffixes=[None,'_old'])
jlh_pl_new = gp.merge(jlh_pl_old, table_pl, left_on='PL2021_ID', right_on='pl',suffixes=[None,'_new'])


#MENGHITUNG SKOR JASLING LAMA
jlh['SPgAir_old'] = jlh['sbava_pg'] + (jlh['PLPg_old']*0.6)
jlh['SPyAir_old'] = jlh['sbava_py'] + (jlh['PLPy_old']*0.6)

#MENGHITUNG SKOR JASLING LAMA
jlh['SPgAir_new'] = jlh['sbava_pg'] + (jlh['PLPg_new']*0.6)
jlh['SPyAir_new'] = jlh['sbava_py'] + (jlh['PLPy_new']*0.6)

#KLASIFIKASI SKOR JASLING(Sangat Rendah - Sangat Tinggi)
class_pg_old = [
    (jlh['SPgAir_old'] <= 1.8),
    (jlh['SPgAir_old'] > 1.8) & (jlh['SPgAir_old'] <= 2.6),
    (jlh['SPgAir_old'] > 2.6) & (jlh['SPgAir_old'] <= 3.4),
    (jlh['SPgAir_old'] > 3.4) & (jlh['SPgAir_old'] <= 4.2),
    (jlh['SPgAir_old'] > 4.2)
    ]

class_py_old = [
    (jlh['SPyAir_old'] <= 1.8),
    (jlh['SPyAir_old'] > 1.8) & (jlh['SPyAir_old'] <= 2.6),
    (jlh['SPyAir_old'] > 2.6) & (jlh['SPyAir_old'] <= 3.4),
    (jlh['SPyAir_old'] > 3.4) & (jlh['SPyAir_old'] <= 4.2),
    (jlh['SPyAir_old'] > 4.2)
    ]

class_pg_new = [
    (jlh['SPgAir_new'] <= 1.8),
    (jlh['SPgAir_new'] > 1.8) & (jlh['SPgAir_new'] <= 2.6),
    (jlh['SPgAir_new'] > 2.6) & (jlh['SPgAir_new'] <= 3.4),
    (jlh['SPgAir_new'] > 3.4) & (jlh['SPgAir_new'] <= 4.2),
    (jlh['SPgAir_new'] > 4.2)
    ]

class_py_new = [
    (jlh['SPyAir_new'] <= 1.8),
    (jlh['SPyAir_new'] > 1.8) & (jlh['SPyAir_new'] <= 2.6),
    (jlh['SPyAir_new'] > 2.6) & (jlh['SPyAir_new'] <= 3.4),
    (jlh['SPyAir_new'] > 3.4) & (jlh['SPyAir_new'] <= 4.2),
    (jlh['SPyAir_new'] > 4.2)
    ]

values = ['Sangat Rendah', 'Rendah', 'Sedang', 'Tinggi', 'Sangat Tinggi']


jlh['PgAir_old'] = np.select(class_pg_old, values)
jlh['PyAir_old'] = np.select(class_py_old, values)
jlh['PgAir_new'] = np.select(class_pg_new, values)
jlh['PyAir_new'] = np.select(class_py_new, values)

# MENYIMPAN HASIL FILE
# now = datetime.now()
# date = now.strftime("%d%m%Y%H%M%S")
# url_file = '/content/drive/MyDrive/Colab Notebooks/result/Jasling/Jasling_'+date+'.shp'
# jlh.to_file(url_file,driver='Shapefile')


# MENAMPILKAN HASIL
# m = leafmap.Map()
# m.add_vector(jlh, layer_name='Jasling')
# jlh.plot(
#     column='PgAir_new',
#     cmap='RdYlGn'
# )

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 106.0 MB/s eta 0:00:00
